# 明暗タスクデータセット 要約


## 前処理
### Pythonによるファイル統合
心理学実験ソフトウェアのログをedfファイルに書き込む

In [ ]:
import os
from labedf import csv2,edf2,set2
import numpy as np
from scipy import signal
from pyedflib import EdfReader
from utils import edf as myedf,edflab as myedflab,signals_standardization,lowpass_filter
DATASET_DIR_PATH = "./dataset/lord2/train"
file_settings = myedflab.MergeAllCsv2EdfFileSettings(DATASET_DIR_PATH + "/ペア.csv",list_encoding="ansi")
edfcsv_filenames = file_settings.get_edfcsv_filenames()
with EdfReader(f"{DATASET_DIR_PATH}/edf/{edfcsv_filenames[0,0]}") as er:
    fs = int(myedf.get_fs(er))

# %% merge csv,edf
file_settings.build_dir_path = f"{file_settings.root_path}/build/1"
filenames = myedflab.merge_all_csv2edf(file_settings,label_header_name="LorD",marker_names=["Marker"],marker_offset=None)
file_settings.build_dir_path = f"{file_settings.root_path}/build/2"
_ = myedflab.merge_all_csv2edf(file_settings,label_header_name="LorD",marker_names=["Wait"],marker_offset=None)

### MATLABで前処理
+ [コード - コミット固定](https://github.com/s-n-1-0/sist_eeg_tests/blob/64c722b010dc63870cb3341b225c3c5873d027e8/notes/dataset/lord2/preprocessing.m)  
(変更点 : エポックリジェクション前の途中経過を保存する処理を加えた、待機時脳波も前処理対象に加えた)

## データセットサイズ

In [ ]:
import mne
labels = ["dark","light"]
before_wait_dataset = []
before_lord_dataset = []
after_wait_dataset = []
after_lord_dataset = []
# 刺激脳波
for filename in filenames:
    epochs = mne.io.read_epochs_eeglab(DATASET_DIR_PATH +"/snapshot/1/"+filename+".set")
    _ld = []
    for label in labels:
        _ld.append(epochs.get_data(item=f"Marker__{label}"))
    before_lord_dataset.append(_ld)
    epochs = mne.io.read_epochs_eeglab(DATASET_DIR_PATH +"/pre2/1/"+filename+".set")
    _ld = []
    for label in labels:
        _ld.append(epochs.get_data(item=f"Marker__{label}"))
    after_lord_dataset.append(_ld)

# 待機脳波
for filename in filenames:
    epochs = mne.io.read_epochs_eeglab(DATASET_DIR_PATH +"/snapshot/2/"+filename+".set")
    _wd = []
    for label in labels:
        _wd.append(epochs.get_data(item=f"Wait__{label}"))
    before_wait_dataset.append(np.concatenate(_wd)) # dark,lightにこだわりがないため結合
    epochs = mne.io.read_epochs_eeglab(DATASET_DIR_PATH +"/pre2/2/"+filename+".set")
    _wd = []
    for label in labels:
        _wd.append(epochs.get_data(item=f"Wait__{label}"))
    after_wait_dataset.append(np.concatenate(_wd))



In [ ]:
import matplotlib.pyplot as plt

before_wait_count = sum([wd.shape[0] for wd in before_wait_dataset])
before_dark_count = sum([ld[0].shape[0]   for ld in before_lord_dataset])
before_light_count = sum([ld[1].shape[0]   for ld in before_lord_dataset])

after_wait_count = sum([wd.shape[0] for wd in after_wait_dataset])
after_dark_count = sum([ld[0].shape[0]   for ld in after_lord_dataset])
after_light_count = sum([ld[1].shape[0]   for ld in after_lord_dataset])


print(f"前処理前 待機画像数 : {before_wait_count}")
print(f"前処理前 刺激画像数(暗) : {before_dark_count}")
print(f"前処理前 刺激画像数(明) : {before_light_count}")
print("------")
print(f"前処理後 待機画像数 : {after_wait_count}")
print(f"前処理後 刺激画像数(暗) : {after_dark_count}")
print(f"前処理後 刺激画像数(明) : {after_light_count}")

plt.bar(["wait"] + labels,[before_wait_count,before_dark_count,before_light_count])
plt.title("Before")
plt.show()
plt.title("After")
plt.bar(["wait"] + labels,[after_wait_count,after_dark_count,after_light_count])


## 加算平均・標準偏差・標準誤差・パワースペクトル(平均)

In [ ]:
from matplotlib.axes import Axes
from scipy.fft import fft
from scipy import signal

t_range = [i/500 - 1 for i in range(1500)]
number =1024 #サンプル数
dt= (number/500)/number #時間幅
freq = np.fft.fftfreq(number,d=dt)
def plot_all_ch(title,plot_func):
    fig = plt.figure(figsize=(30,8),facecolor="white")
    fig.suptitle(title)
    plt.subplots_adjust(wspace=0.4, hspace=0.6)
    for i in range(10):
        ax = fig.add_subplot(3, 4, i+1)
        plot_func(title,ax,i)
        ax.set_title(f"{i + 1}ch")
        ax.legend(loc='upper left', bbox_to_anchor=(1,1))
        ax.axvline(x=0,color="gray")
    return fig

def save_all_ch(path:str,title:str,data:np.ndarray,is_fft:bool = False):
    fig = plot_all_ch(title,data,is_fft=is_fft)
    fig.savefig(path)
    plt.close(fig)
class PlotMan():
    def __init__(self,is_before:bool,_dataset:list[np.ndarray],dataset_type:str) -> None:
        dataset = np.concatenate(_dataset)
        l = dataset.shape[0]
        erp = np.sum(dataset,axis=0) / l
        sum_eeg = np.sum(dataset,axis=0)
        std_eeg = np.std(dataset,axis=0,ddof=1)
        se_eeg = std_eeg /np.sqrt(l)
        psd = np.concatenate(_dataset)[:,:,750-512:750+512]
        han = signal.hann(number)
        for i in range(psd.shape[0]):
            for j in range(psd.shape[1]):
                psd[i,j,:] *= han
        psd_eeg = np.abs(fft(psd,axis=-1)) ** 2
        psd_eeg = np.sum(psd_eeg,axis=0) / l
        self.erp = erp
        self.sum_eeg = sum_eeg
        self.std_eeg = std_eeg
        self.se_eeg = se_eeg
        self.is_before = is_before
        self.dataset_type = dataset_type
        self.psd_eeg = psd_eeg
    
    def plot_sum(self):
        def _plot_sum(_,i):
            plt.plot(t_range,self.plot_sum[i,:])
        plot_all_ch(f"[{'Before' if self.is_before else 'After'}] All {self.dataset_type.capitalize()} Sum",_plot_sum)
    def plot_psd(self):
        def plot_all_psd(title):
            fig = plt.figure(figsize=(30,8),facecolor="white")
            fig.suptitle(title)
            plt.subplots_adjust(wspace=0.4, hspace=0.6)
            for i in range(10):
                psd = self.psd_eeg
                plt.plot(freq[1:150],psd[i,1:150],label=i+1)
                #ax.set_title(f"{i + 1}ch")
            plt.legend(loc='upper left', bbox_to_anchor=(1,1))
            plt.ylim(0,10**(-4.9))
            return fig
            
        plot_all_psd(f"[{'Before' if self.is_before else 'After'}] All {self.dataset_type.capitalize()} Psd")
before_wait = PlotMan(True,before_wait_dataset,"wait")
before_dark = PlotMan(True,[ld[0] for ld in before_lord_dataset],"dark")
before_light = PlotMan(True,[ld[1] for ld in before_lord_dataset],"light")

after_wait =  PlotMan(False,after_wait_dataset,"wait")
after_dark = PlotMan(False,[ld[0] for ld in after_lord_dataset],"dark")
after_light = PlotMan(False,[ld[1]   for ld in after_lord_dataset],"light")

before_plots = [before_wait,before_dark,before_light]
after_plots = [after_wait,after_dark,after_light]


### ERP-SE

In [ ]:
def plot_erp_se(is_before:bool):
    def _plot_erp_se(title:str,ax:Axes,i:int):
        def __plot_erp_se(erp,se_eeg,data_type:str):
            erp = erp[i,:]
            se = se_eeg[i,:]
            p = ax.plot(t_range,erp+se,label=data_type.capitalize(),linestyle="dashed")
            ax.plot(t_range,erp-se,label=f"_{data_type.capitalize()}",color=p[0].get_color(),linestyle="dashed")
            ax.plot(t_range, erp,label=f"_{data_type.capitalize()}",color=p[0].get_color())
        if is_before:
            for bp in before_plots:
                __plot_erp_se(bp.erp,bp.se_eeg,bp.dataset_type)
        else:
            for ap in after_plots:
                __plot_erp_se(ap.erp,ap.se_eeg,ap.dataset_type)
    plot_all_ch(f"[{'Before' if is_before else 'After'}] ERP-SE",_plot_erp_se)

plot_erp_se(True)
plot_erp_se(False)

### ERP-STD

In [ ]:
def plot_erp_std(is_before:bool):
    def _plot_erp_std(title:str,ax:Axes,i:int):
        def __plot_erp_std(erp,std_eeg,data_type:str):
            erp = erp[i,:]
            std = std_eeg[i,:]
            p = ax.plot(t_range,erp+std,label=data_type.capitalize(),linestyle="dashed")
            ax.plot(t_range,erp-std,label=f"_{data_type.capitalize()}",color=p[0].get_color(),linestyle="dashed")
            ax.plot(t_range, erp,label=f"_{data_type.capitalize()}",color=p[0].get_color())
        if is_before:
            for bp in before_plots:
                __plot_erp_std(bp.erp,bp.std_eeg,bp.dataset_type)
        else:
            for ap in after_plots:
                __plot_erp_std(ap.erp,ap.std_eeg,ap.dataset_type)
    plot_all_ch(f"[{'Before' if is_before else 'After'}] ERP-STD",_plot_erp_std)

plot_erp_std(True)
plot_erp_std(False)

### PS

In [ ]:
for p in (before_plots + after_plots):
    p.plot_psd()